# 基于T5的文本摘要

## Step1 导入相关包

In [1]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

d:\anaconda3\envs\py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./data/nlpcc_2017/")
ds

Dataset({
    features: ['title', 'content'],
    num_rows: 5000
})

In [3]:
print(ds[0])

{'title': '澳大利亚央行将利率降至纪录低点,以应对疲软的经济前景,并遏制澳元进一步走强。', 'content': '澳大利亚央行将利率降至纪录低点,以应对疲软的经济前景,并遏制澳元进一步走强。05/0513:37|评论(0)A+澳大利亚央行周二发布声明称,将关键利率由2.25%调降至2%,符合此前交易员及接受彭博调查的29位经济学家中25位的预期。据彭博社报道,上月澳央行官员曾警告,矿业之外的行业投资可能下滑。澳大利亚政府不太可能推出新的刺激措施,来扶助受本币升值和铁矿石价格下跌打击而低于潜在水平的经济增长。“鉴于大宗商品价格下跌,矿业投资还可能有低于当前预期的风险,”预计到降息的澳新银行高级经济学家FelicityEmmett在决议公布前编写的研究报告中称。他表示此次决议可能反映出“央行经济增长预估轨迹有所下调”。'}


In [4]:
ds = ds.train_test_split(test_size=0.2)
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['title', 'content'],
        num_rows: 1000
    })
})

## Step3 数据处理

In [7]:
# 数据处理的核心是，把输入和标签对应上
'''
{
    'inputs':[],
    'labels':[]
}
'''

tokenizer = AutoTokenizer.from_pretrained('D:/pretrained_model/models--Langboat--mengzi-t5-base')

def process_function(examples):
    contents = ["根据上下文进行文本摘要：\n" + e for e in examples["content"]]
    inputs = tokenizer(contents, max_length = 384, truncation=True)
    labels = tokenizer(examples['title'], max_length=100, truncation=True)
    inputs['labels'] = labels['input_ids']
    return inputs

In [8]:
tokenizer_dataset = ds.map(process_function, batched=True)
tokenizer_dataset

Map: 100%|██████████| 1000/1000 [00:00<00:00, 4280.89 examples/s]


DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [21]:
print(tokenizer_dataset['train'][1]['labels'])

[7, 1027, 126, 512, 5386, 7850, 229, 1443, 1443, 1559, 628, 10751, 21022, 2249, 218, 50, 18358, 3, 4242, 19405, 201, 28, 2236, 768, 1890, 218, 440, 3, 247, 1481, 10297, 1236, 23940, 4, 1]


In [14]:
print(tokenizer.decode(tokenizer_dataset['train'][1]['input_ids']))

根据上下文进行文本摘要: 我国最北高寒高铁线路--哈尔滨至齐齐哈尔客运专线23日开始联调联试,预计8月正式通车,届时中国高铁将向北延伸281公里。哈齐客专是我国纬度最高的高寒高铁线路,是“哈大齐工业走廊”的重要通道,也是黑龙江省内第一条城际客运专线。哈齐客专工程于2009年11月30日正式开工建设,投资概算总额323.9亿元,新建正线长度281公里,桥梁占正线里程的61.7%,双线电气化,设计速度250公里/小时,全线共设哈尔滨北、肇东、安达、大庆东、大庆西、泰康、红旗营东和齐齐哈尔南8个车站。哈齐客专与哈大高铁直接相通,有望成为连接黑龙江省内与省外大中城市的快速通道和主要干道,为“中蒙俄经济走廊”通道建设等提供铁路交通支撑。据哈尔滨铁路局工作人员介绍,本次联调联试范围为哈尔滨北站至齐齐哈尔南站,主要是综合检测列车和相关线路设备,在规定测试速度下对全线各系统进行综合调试,评价和验证供变电、接触网、通信、信号、客服、自然灾害及异物侵限监测等系统功能以及路基、轨道、道岔、桥梁等结构工程的适用性,使各系统功能达到设计要求,为全线顺利开通运营提供科学依据。(记者邹大鹏、王君宝)</s>


In [15]:
print(tokenizer.decode(tokenizer_dataset['train'][1]['labels']))

我国最北高铁哈尔滨至齐齐哈尔客运专线预计8月通车,途径绥化大庆共设8站,设计速度250公里每小时。</s>


## Step4 创建模型

In [16]:
model = AutoModelForSeq2SeqLM.from_pretrained('D:/pretrained_model/models--Langboat--mengzi-t5-base')

## Step5 创建评估函数

In [22]:
tokenizer.pad_token_id

0

In [24]:
import numpy as np
from rouge_chinese import Rouge

rouge = Rouge()

def compute_metric(evalPred):
    predition, labels = evalPred
    decode_pred = tokenizer.decode(predition, skip_special_tokens=True)
    __labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
    docode_labels = tokenizer.batch_decode(__labels,skip_special_tokens=True)
    decode_preds = [" ".join(p) for p in decode_pred]
    decode_labels = [" ".join(l) for l in docode_labels]
    scores = rouge.get_scores(decode_preds, decode_labels, avg=True)
    return {
        "rouge-1": scores["rouge-1"]["f"],
        "rouge-2": scores["rouge-2"]["f"],
        "rouge-l": scores["rouge-l"]["f"],
    }


## Step6 配置训练参数

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir="./summary",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    logging_steps=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="rouge-l",
    predict_with_generate=True
)

## Step7 创建训练器

In [ ]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    train_dataset=tokenizer_dataset["train"],
    eval_dataset=tokenizer_dataset["test"],
    compute_metrics=compute_metric,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer)
)

## Step8 模型训练

In [ ]:
trainer.train()

## Step9 模型推理

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0)

In [ ]:
pipe("摘要生成:\n" + ds["test"][-1]["content"], max_length=64, do_sample=True)

In [ ]:
ds["test"][-1]["title"]